In [1]:
print("Hello World")

Hello World


In [2]:
import pyodbc

# ----------------------------
# STEP 1: Define your connection
# ----------------------------
server = r'(localdb)\MSSQLLocalDB'    # local SQL Server instance
database = 'Arsipa'             # make sure the name matches exactly

# define conn_str for Windows Authentication for SQL Server

conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    f"Server={server};"
    f"Database={database};"
    "Trusted_Connection=yes;"
    "Encrypt=no;"
)

# ----------------------------
# STEP 2: Connect to SQL Server
# ----------------------------
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connection to SQL Server successful!")
except Exception as e:
    print("❌ Failed to connect to SQL Server:")
    print(e)
    conn = None

# ----------------------------
# STEP 3: Run a simple query
# ----------------------------
if conn:
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT @@VERSION;")  # Returns SQL Server version
        for row in cursor.fetchall():
            print("SQL Server Version:", row[0])
    except Exception as e:
        print("❌ Query failed:", e)
    finally:
        conn.close()
        print("🔒 Connection closed.")


✅ Connection to SQL Server successful!
SQL Server Version: Microsoft SQL Server 2019 (RTM-CU27-GDR) (KB5040948) - 15.0.4382.1 (X64) 
	Jul  1 2024 20:03:23 
	Copyright (C) 2019 Microsoft Corporation
	Express Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 26100: ) (Hypervisor)

🔒 Connection closed.


In [23]:
import pandas as pd

# Load CSV Data files
df_umsatz = pd.read_csv(r"C:\Users\Dell\Desktop\Arsipa Case Study\umsatz.csv")
df_mitarbeiter = pd.read_csv(r"C:\Users\Dell\Desktop\Arsipa Case Study\mitarbeiter.csv")
df_gesellschaften = pd.read_csv(r"C:\Users\Dell\Desktop\Arsipa Case Study\gesellschaften.csv")

# View Dataset
df_umsatz.head(5)
# df_mitarbeiter.tail(5)
# df_gesellschaften.head(5)




,gesellschaft_id,monat,umsatz_eur
0,98,2023-06,214456
1,70,2023-11,228200
2,86,2023-07,121279
3,11,2023-01,475676
4,16,2023-05,168280


In [51]:
# Joining Fact Table with Dim(s)
fact = ( df_umsatz.merge(df_mitarbeiter, on=["gesellschaft_id", "monat"], how="left")
                                        .merge(df_gesellschaften, on="gesellschaft_id", how="left")
       )

# Example: KPI "Sales_per_Employee"
fact["umsatz_pro_mitarbeiter"] = fact["umsatz_eur"] / fact["anzahl_mitarbeiter"].replace(0, pd.NA)

# View Fact Table after Joins
fact 

# Group by gesellschaft_id and calculate SUM of umsatz_pro_mitarbeiter
result_sum = fact.groupby("gesellschaft_id")["umsatz_pro_mitarbeiter"].sum()

# Group by gesellschaft_id and calculate MEAN (average)
result_mean = fact.groupby(["gesellschaft_id", "monat"])["umsatz_pro_mitarbeiter"].mean()

# If you want both at once
result = fact.groupby("gesellschaft_id")["umsatz_pro_mitarbeiter"].agg(["sum", "mean", "max", "min"])

print(result_mean.head(20))


gesellschaft_id  monat  
1                2023-01            NaN
                 2023-02            NaN
                 2023-03     328.945175
                 2023-04     181.173596
                 2023-06     109.219677
                 2023-08            NaN
                 2023-09            NaN
                 2023-10    1959.746479
                 2023-11     275.412899
2                2023-01     963.892865
                 2023-07            NaN
                 2023-08            NaN
                 2023-10     654.731595
                 2023-11            NaN
                 2023-12            NaN
3                2023-01            NaN
                 2023-02     191.862419
                 2023-03            NaN
                 2023-04     470.922822
                 2023-07     627.418046
Name: umsatz_pro_mitarbeiter, dtype: float64
